<p id="back" style="background-color:powderblue; font-size: 200%; color: red;text-align:center;"> 回到这里</p>

$$
\text{自定义命令}  
\newcommand{\inner}[2]{{\langle #1,#2\rangle}}
\newcommand{\dlim}{\displaystyle \lim}  
\newcommand{\dint}{\displaystyle \int}   
\newcommand{\ra}{\rangle}  
\newcommand{\la}{\langle}   
\newcommand{\inner}[2]{{\langle #1,#2\rangle}}
\newcommand{\x}{\mathbf{x}}   
\newcommand{\xt}{\mathbf{x}^{\mathsf{T}}}   
\newcommand{\T}{{\mathsf{T}}}
\newcommand{\abf}{\mathbf{a}}   
\newcommand{\abft}{\mathbf{a}^{\mathsf{T}}}   
\newcommand{\R}{\mathbb{R}}   
\newcommand{\C}{\mathbb{C}}   
\newcommand{\E}{\mathrm{e}}   
\newcommand{\F}{\mathbb{F}}   
\newcommand{\X}{\mathbf{X}}   
\newcommand{\Y}{\mathbf{Y}}  
\newcommand{\f}{\mathbf{f}}  
\newcommand{\U}{\mathbf{u}}   
\newcommand{\D}{\mathrm{d}}  
\newcommand{\M}{\mathcal{M}}  
\newcommand{\LL}{\mathcal{L}}  
\newcommand{\nullspace}{\mathrm{null}}   
\newcommand{\range}{\mathrm{range}}   
\newcommand{\Sum}[2]{{\sum_{#1}^{#2}}}   
\newcommand{\Union}[2]{{\bigcup_{#1}^{#2}}}   
\newcommand{\Intersection}[2]{{\bigcap_{#1}^{#2}}}   
\newcommand{\pd}[1]{\frac{\partial}{\partial #1}} \notag
$$  

In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
import os 
import matplotlib as mpl
import matplotlib.pyplot as pl

In [3]:
path_default = os.getcwd()
path_figure = path_default + '/figure'
path_data = path_default + '/datasets'
path_result = path_default + '/results'

In [4]:
if not os.path.exists(path_result):   
    os.mkdir(path_result)
if not os.path.exists(path_data):   
    os.mkdir(path_data)
if not os.path.exists(path_figure):   
    os.mkdir(path_figure)

In [6]:
import zsh_in_jupyter as zsh
zsh.opendir()

<h1><a href="#back">调仓频率</a></h1>

In [ ]:
pd.DataFrame.plot()

<h1>策略评判指标</h1>

## 收益率指标

- 简单收益率:
$$
简单收益率 = \frac{V_T-V_0}{V_0}, 年华后 = (\frac{V_T}{V_0})^{\frac{1}{T}-1}
$$
- continuous compound rate of return
$$
\ln(\frac{V_T}{V_0}),\ 年化后 \frac{1}{T}\ln(\frac{V_T}{V_0})
$$

- 对于债券和存款，通常一年选用365，股票期权等选用252

## 风险调整收益率

- beta:
$$
\beta_p = \frac{cov(r_p,r_M)}{var(r_M)}
$$
beta衡量了现有投资组合对于市场的反应程度，beta=1代表市场中性策略，不是beta越大越好，只是一个相关性的度量
- Jensen's alpha:
$$
单因子模型中的\alpha = E(r_p)-r_f-\beta(E(r_B)-r_f)，衡量该策略超出市场的收益
$$
衡量股票的特质风险，投资者获取与市场波动无关的收益，$\alpha$越高越好，量化投资就是在不断提升alpha的过程
- **Sharpe ratio**:
$$
    SP = \frac{E(r_p)-r_f}{\sigma_p}
$$
表示每承受一单位风险，会产生多少的超额报酬，可以同时对策略的收益与风险进行综合考虑。越大越好！
- Treynor ratio:
$$
    Tn = \frac{E(r_p)-r_f}{\beta_p}
$$
where $\beta_p=\frac{Cov(r_p,r_M)}{Var(r_M)}$

- information ratio:
$$
\frac{\alpha}{\sigma_{\epsilon}}
$$
信息比率大的好

## 其它指标

### 累计收益率曲线(profit and loss)

假设投资者的初始资金是 V 0 ，该投资者在 t 时刻的资金总量为 Vt，那么 Vt 关于 t 的函数图像 就叫作累积收益曲线即 P&L曲线。P&L曲线是最直观的评判投资绩效的标准，这里 P&L表示 损益（profit and loss）。一个量化投资策略如果表现优异，其对应的 P&L 曲线应当尽可能保持 具有平稳上升的走势。


### 最大回撤

- drawdown $DD_t$: 最大回撤
$$
DD_t = \max(\max_{s\in[0,t]}V_s-V_t,0)
$$
- Max Drawdown:
$$
MDD_t = \max_{i<j\leq t}(1-\frac{P_j}{P_i})
$$
- 0到T时刻的最大回撤定义为:
$$
MDD_T = \max_{s \in [0,T]}DD_t
$$
最大回撤越小越好，但其实即便是指数的最大回撤，可能都有百分之30

### 换手率

- turnover rate:  
假设投资者投资于 $n$ 个资产, 在 $t$ 时刻进行调仓操作 (即买卖资产, 调整各个资产持有量), 调 仓前持有资产总价值为 $V_t$, 其中各个资产价值分别为 $V_{t-}^1, V_{t-}^2, \ldots, V_{t-}^n$; 调仓后持有的各个资 产价值分别变为 $V_{t+}^1, V_{t+}^2, \ldots, V_{t+}^n$ 。则 $t$ 时刻的换手率被定义为
$$
换手率_t=\frac{\sum_{i=1}^n\left|V_{t-}^i-V_{t+}^i\right|}{2 V_t}.
$$
换手率衡量的是调仓前后持仓的差异，用于测算交易成本，如果一个策略的换手率较高，其调仓时的交易成本就会较大
年化换手率:
$$
\sum_
$$

# 案例分析

In [4]:
ls1 = pd.Series([1,2,3,4])
ls1.mean()

float

In [5]:
from jqdatasdk import *
auth('18323330059','Zny1780068440')  #账号是申请时所填写的手机号；密码为聚宽官网登录密码

auth success 


In [11]:
stocks = list(get_all_securities(['stock']).index)

In [14]:
stocks[0]

'000001.XSHE'

In [21]:
df = get_price('000001.XSHE', end_date='2022-01-30 14:00:00',count=40, frequency='1m', 
               fields=['open','close','high','low','volume','money'])
df

,open,close,high,low,volume,money
2022-01-28 14:21:00,15.72,15.71,15.72,15.71,327388.0,5144279.0
2022-01-28 14:22:00,15.72,15.72,15.73,15.71,438349.0,6889019.0
2022-01-28 14:23:00,15.73,15.72,15.73,15.72,268548.0,4222094.0
2022-01-28 14:24:00,15.73,15.73,15.73,15.72,463596.0,7290256.0
2022-01-28 14:25:00,15.74,15.74,15.74,15.72,986337.0,15508035.0
2022-01-28 14:26:00,15.72,15.72,15.74,15.72,722778.0,11366181.0
2022-01-28 14:27:00,15.73,15.72,15.74,15.72,688573.0,10825146.0
2022-01-28 14:28:00,15.72,15.72,15.73,15.72,956408.0,15035770.0
2022-01-28 14:29:00,15.73,15.73,15.73,15.72,346526.0,5447993.0
2022-01-28 14:30:00,15.72,15.72,15.73,15.72,860004.0,13520206.0


5.3 策略风险评价指标


In [24]:
a = pd.Series([1,2,3],index=['a','b','c'])
a

a    1
b    2
c    3
dtype: int64